In [1]:
# !pip install scikit-learn

In [9]:
import openeo
import scipy
import numpy as np
from openeo.api.process import Parameter
from openeo.extra.spectral_indices import compute_indices


connection = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


In [10]:
# define input parameter
spatial_extent = Parameter.spatial_extent(
    name="spatial_extent", 
    description="Limits the data to process to the specified bounding box or polygons.\\n\\nFor raster data, the process loads the pixel into the data cube if the point at the pixel center intersects with the bounding box or any of the polygons (as defined in the Simple Features standard by the OGC).\\nFor vector data, the process loads the geometry into the data cube if the geometry is fully within the bounding box or any of the polygons (as defined in the Simple Features standard by the OGC). Empty geometries may only be in the data cube if no spatial extent has been provided.\\n\\nEmpty geometries are ignored.\\nSet this parameter to null to set no limit for the spatial extent."
    )

temporal_extent = Parameter.temporal_interval(
    name="temporal_extent", 
    description="Temporal extent specified as two-element array with start and end date/date-time."
    # default=["2023-05-01", "2023-07-30"],
    )

In [11]:
def get_cloud_free_pixels(scl, data, reducer="median"):
    mask = (scl == 3) | (scl == 8) | (scl == 9) | (scl == 10)

    # mask is a bit noisy, so we apply smoothening
    # 2D gaussian kernel
    g = scipy.signal.windows.gaussian(11, std=1.6)
    kernel = np.outer(g, g)
    kernel = kernel / kernel.sum()

    # Morphological dilation of mask: convolution + threshold
    mask = mask.apply_kernel(kernel)
    mask = mask > 0.1

    data_masked = data.mask(mask)

    # now select Best Available Pixel based on the mask
    return data_masked.reduce_dimension(reducer=reducer, dimension="t")

In [12]:
# preprocessing in openEO
def prep_s2(date,aoi,reducer):
    # load S2 pre-collection
    s2_cube = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=date,
        spatial_extent=aoi,
        bands=["B02", "B03", "B04", "B08","B12"],
        max_cloud_cover=80,
    )
    
    s2_scl = connection.load_collection(
        "SENTINEL2_L2A",
        temporal_extent=date,
        spatial_extent=aoi,
        bands=["SCL"],
        max_cloud_cover=80,
    )
    
    # Create a Pre-event cloud-free mosaic
    cf_cube = get_cloud_free_pixels(s2_scl, s2_cube, reducer=reducer)

    # calculate all indices
    # what are the indices we need?
    #== NBR NDFI Contrast GLCM Variance, Mean, Correlation, BAI, Angular moment, dissimilarity, homogenity
    
    # nbr = compute_indices(bap_cube, indices=["NBR"])
        # compute the spectral indices
    indices_list = ["NBR", "BAI"]
    indices = compute_indices(cf_cube, indices_list)

    # calculate texture features
    features_udf = openeo.UDF.from_file("features.py")
    features = cf_cube.apply_neighborhood(
        process=features_udf,
        size=[
            {"dimension": "x", "value": 128, "unit": "px"},
            {"dimension": "y", "value": 128, "unit": "px"},
        ],
        overlap=[
            {"dimension": "x", "value": 0, "unit": "px"},
            {"dimension": "y", "value": 0, "unit": "px"},
        ],
    )
    features = features.rename_labels(dimension = "bands", target = ["contrast","homogeneity","ASM","variance","NDFI"])
    
    # combine the original bands with the computed indices
    combined = cf_cube.merge_cubes(indices)
    combined = combined.merge_cubes(features)
    return combined

In [13]:
s2_feature_cube_in = prep_s2(temporal_extent,spatial_extent,"median")


In [14]:
model = "https://openeo.vito.be/openeo/1.2/jobs/j-25052516461141e0bbcca95d9541b623/results/items/ZWNjZTlmZWEwNGI4YzljNzZhYzc2YjQ1YjZiYTAwYzIwZjIxMWJkYTQ4NTZjMTRhYTQ0NzViOGU4ZWQ0MzNjZEBlZ2kuZXU=/da32856ccabf0e20b615da06941898b4/ml_model_metadata.json?expires=1748797341"

In [15]:
inference = s2_feature_cube_in.predict_random_forest(
    model=model,
    dimension="bands"
)

In [16]:
udp_randomfire = connection.save_user_defined_process(
    user_defined_process_id="random_fire",
    process_graph=inference,
    parameters=[temporal_extent, spatial_extent],
    summary="Testing Forest fire",
    description="testing random forest model",
    public=True,
)

Preflight process graph validation raised: [ProcessParameterRequired] Process 'n/a' parameter 'spatial_extent' is required.


In [18]:
udp_randomfire

In [ ]:
public_url = "https://openeo.vito.be/openeo/1.2/processes/u:ecce9fea04b8c9c76ac76b45b6ba00c20f211bda4856c14aa4475b8e8ed433cd@egi.eu/random_fire"

In [22]:
# reuse the model directly in the webeditor:
job_id = "j-2505251437454a3db3af475c7f74bc3a"

{
  "process_graph": {
    "random1": {
      "arguments": {
        "spatial_extent": {
          "east": -118.10448985345741,
          "north": 34.22527587881902,
          "south": 34.206163644370264,
          "west": -118.14752510292374
        },
        "temporal_extent": [
          "2025-01-01T00:00:00Z",
          "2025-01-15T00:00:00Z"
        ]
      },
      "process_id": "random_fire",
      "result": true
    }
  }
}